In [1]:
# Load timeseries

import pickle

tsfile = '/run/media/anton/Samsung_T5/BSNIP/bsnip_rc_ts_centered.pkl'

ts = pickle.load(open(tsfile, 'rb'))

print(ts.keys())

dict_keys(['Baltimore_S0015SRH1', 'Baltimore_S0021FHS1', 'Baltimore_S0109FAN1', 'Baltimore_S0132MTD1', 'Baltimore_S0172LAC1', 'Baltimore_S0186SQN1', 'Baltimore_S0237KSC1', 'Baltimore_S0269ICH1', 'Baltimore_S0272EQV1', 'Baltimore_S0314HOW1', 'Baltimore_S0316VHT1', 'Baltimore_S0369KXN1', 'Baltimore_S0371FIY1', 'Baltimore_S0424PGG1', 'Baltimore_S0437DNJ1', 'Baltimore_S0486PLY1', 'Baltimore_S0557YAU1', 'Baltimore_S0686YET1', 'Baltimore_S0699JUW1', 'Baltimore_S0804JJR1', 'Baltimore_S0828ICR1', 'Baltimore_S0833NUO1', 'Baltimore_S0837DFF1', 'Baltimore_S0898PVW1', 'Baltimore_S0902ULJ1', 'Baltimore_S0926USC1', 'Baltimore_S0935JNE1', 'Baltimore_S0952PBW1', 'Baltimore_S0958KLO1', 'Baltimore_S1006VGG1', 'Baltimore_S1020YFA1', 'Baltimore_S1033ONJ1', 'Baltimore_S1044BWP1', 'Baltimore_S1109FMB1', 'Baltimore_S1109WGJ1', 'Baltimore_S1132EWE1', 'Baltimore_S1134EQT1', 'Baltimore_S1222ERE1', 'Baltimore_S1304MCJ1', 'Baltimore_S1395AEC1', 'Baltimore_S1452MLX1', 'Baltimore_S1467CBC1', 'Baltimore_S1540SEW1', 

In [3]:
# Get sz diagnosis and hc

demofile = '/home/anton/Documents/Tulane/Research/ImageNomer/data/anton/cohorts/BSNIP/demographics.pkl'

demo = pickle.load(open(demofile, 'rb'))

print(demo.keys())
print(demo['DXGROUP_1'].keys())

dict_keys(['DXGROUP_1', 'DXGROUP_2', 'sex', 'ethnic', 'Race', 'handed', 'MED_PSYCHOTROPIC', 'MED_ANTIP_ANY_PRI', 'MED_ANTIP_FIRSTGEN_SEC', 'MED_ANTIP_SECONDGEN_SEC', 'MED_ANTID_ANY_PRI', 'MED_ANTID_TRICYCLIC_SEC', 'MED_ANTID_MAOI_SEC', 'MED_ANTID_SSRI_SEC', 'MED_ANTID_MISC_SEC', 'MED_MOOD_ANY_PRI', 'MED_MOOD_LITHIUM_SEC', 'MED_MOOD_ANTICONVUL_SEC', 'MED_ANX_HYP_PRI', 'MED_ANTICHOL_PRI', 'MED_CENT_ACTIVE_PRI', 'MED_STIMULANTS_PRI', 'Age_cal', 'I2a', 'PANSS_PosTotal', 'PANSS_NegTotal', 'PANSS_GenTotal', 'PANSS_Total', 'pt_hollscore', 'mo_hollscore', 'fa_hollscore', 'Family_hollscore', 'sz_subtype', 'sad_subtype', 'bp_subtype'])
dict_keys(['Baltimore.S0015SRH1', 'Baltimore.S0021FHS1', 'Baltimore.S0172LAC1', 'Baltimore.S0186SQN1', 'Baltimore.S0237KSC1', 'Baltimore.S0269ICH1', 'Baltimore.S0314HOW1', 'Baltimore.S0316VHT1', 'Baltimore.S0369KXN1', 'Baltimore.S0371FIY1', 'Baltimore.S0424PGG1', 'Baltimore.S0437DNJ1', 'Baltimore.S0486PLY1', 'Baltimore.S0557YAU1', 'Baltimore.S0686YET1', 'Baltimore

In [13]:
sz_scans = []
nc_scans = []

for sub, val in demo['DXGROUP_1'].items():
    if val == 'SZP':
        sz_scans.append(ts[sub.replace('.', '_')])
    if val == 'NC':
        nc_scans.append(ts[sub.replace('.', '_')])
    
print(len(sz_scans), len(nc_scans))

199 243


In [30]:
# Choose 25 SZ and 25 NC

from random import sample
import numpy as np
from sklearn.linear_model import LogisticRegression

N = 25
W = 50

errs = []

for _ in range(20):
    sz = sample(sz_scans, k=N)
    nc = sample(nc_scans, k=N)

    print(len(sz))
    print(sz[0].shape)

    szw = []
    ncw = []

    # Convert to windowed DFC

    a,b = np.triu_indices(264,1)

    for i in range(N):
        for grp, grpw in zip([sz, nc], [szw, ncw]):
            grpw.append([])
            for j in range(0, grp[i].shape[1], W):
                slc = grp[i][:,j:j+W]
                if slc.shape[1] < W:
                    break
                p = np.corrcoef(slc)[a,b]
                grpw[-1].append(p)
            grpw[-1] = np.stack(grpw[-1])
    #         print(grpw[-1].shape)

    # Choose 1st for train and 2nd for test

    xtr1 = np.stack([dfc[0] for dfc in szw[:25]])
    xtr2 = np.stack([dfc[0] for dfc in ncw[:25]])
    xt1 = np.stack([dfc[1] for dfc in szw[:25]])
    xt2 = np.stack([dfc[1] for dfc in ncw[:25]])

    xtr = np.concatenate([xtr1, xtr2])
    xt = np.concatenate([xt1, xt2])

    ytr = np.concatenate([np.zeros(25), np.ones(25)])
    yt = ytr

    reg = LogisticRegression(max_iter=1000).fit(xtr, ytr)
    yhat = reg.predict(xt)
    err = np.mean(yhat == yt)
    print(err)
    errs.append(err)
    
print(np.mean(errs), np.std(errs))

25
(264, 204)
0.84
25
(264, 204)
0.76
25
(264, 204)


KeyboardInterrupt: 